In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
import sys
project_root = os.path.abspath(os.path.join("/data/fengjw/Project/DAMA/DAMA/"))
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
# from llava.model import *
# import torch
# from llava.model.builder import load_pretrained_model
# from transformers import AutoTokenizer
# from llava.constants import DEFAULT_IMAGE_PATCH_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
# from llava.utils import disable_torch_init
# from llava.mm_utils import tokenizer_image_token, process_images, get_model_name_from_path
# from llava.conversation import conv_templates
# from PIL import Image
# from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
import torch
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
from llava.conversation import conv_templates
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from llava.mm_utils import tokenizer_image_token, process_images, get_model_name_from_path
from PIL import Image
from matplotlib import pyplot as plt
from PIL import Image
import json
from tqdm import tqdm

/NAS/fengjw/miniconda3/envs/OPA_DPO/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model_path = "/NAS/fengjw/.cache/output/DAMA/ablation/llava7b_dpo_reference_free_bate_10/checkpoint-5000" 
model_base = None
disable_torch_init()
model_name = os.path.expanduser(model_path)
model_name = get_model_name_from_path(model_path)

In [7]:
print('Loading RePO...')
tokenizer, repo, image_processor, context_len = load_pretrained_model(model_path, model_base, model_name)

Loading RePO...


Loading checkpoint shards: 100%|██████████| 2/2 [02:16<00:00, 68.13s/it]
Some weights of the model checkpoint at /NAS/fengjw/.cache/output/DAMA/ablation/llava7b_dpo_reference_free_bate_10/checkpoint-5000 were not used when initializing LlavaLlamaForCausalLM: ['model.vision_tower.vision_tower.vision_model.encoder.layers.2.layer_norm2.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.12.self_attn.out_proj.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.11.mlp.fc1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.15.self_attn.v_proj.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.1.layer_norm1.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.16.layer_norm1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.23.self_attn.v_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.13.layer_norm1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.lay

In [8]:
def inference(model, question, image_file):

    qs = DEFAULT_IMAGE_TOKEN + '\n' + question
    conv = conv_templates['v1'].copy()
    conv.append_message(conv.roles[0], qs)
    conv.append_message(conv.roles[1], None)
    prompt = conv.get_prompt()

    image = Image.open(image_file).convert('RGB')
    image_tensor = process_images([image], image_processor, model.config)[0]
    input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()

    
    print(model.dtype) 
    print(input_ids.dtype) 
    print(image_tensor.dtype) 
    
    with torch.inference_mode():
        output_ids = model.generate(
            input_ids,
            images=image_tensor.unsqueeze(0).half().cuda(),
            do_sample=False,
            temperature=0,
            top_p=None,
            num_beams=1,
            max_new_tokens=1024,
            use_cache=True)

    input_token_len = input_ids.shape[1]
    n_diff_input_output = (input_ids != output_ids[:, :input_token_len]).sum().item()
    if n_diff_input_output > 0:
        print(f'[Warning] {n_diff_input_output} output_ids are not the same as the input_ids')
    outputs = tokenizer.batch_decode(output_ids[:, input_token_len:], skip_special_tokens=True)[0]
    outputs = outputs.strip()

    return outputs

In [9]:
question = "Describe the image in detail."
image_file='/NAS/fengjw/Project/eval_data/AMBER/image/AMBER_1.jpg'

In [10]:
outputs = inference(repo, question, image_file)

torch.float16
torch.int64
torch.float32


RuntimeError: mat1 and mat2 must have the same dtype